In [ ]:
import gradio as gr

In [ ]:
def greet(name):
    return f"Hi, {name}"

In [ ]:
ui = gr.Interface(fn = greet, inputs = 'text', outputs = 'text', flagging_mode = 'never')

In [ ]:
ui.launch()

# CHAT UI with Gradio (Two persona)

In [ ]:
from litellm import completion
import gradio as gr

OLLAMA_MODEL = 'ollama_chat/llama3.2'

In [ ]:
SNARKY = """
    You are a chatbot who is very argumentative;
    you disagree with anything in the conversation 
    and you challenge everything, in a snarky way.
    Keep the message short, say 25 words
"""

HUMBLE = """ 
    You are a very polite, courteous chatbot. You try to agree with
    everything the other person says, or find common ground. 
    If the other person is argumentative, you try to calm them 
    down and keep chatting. Keep the messages short, say 25 words
"""

In [ ]:
def ask_snarky(query):
    stream = completion(
        model = OLLAMA_MODEL,
        messages = [
            {"role": "system", "content": SNARKY},
            {"role": "user", "content": query}
        ],
        stream = True
    )

    response = ''
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

def ask_humble(query):
    stream = completion(
        model = OLLAMA_MODEL,
        messages = [
            {"role": "system", "content": HUMBLE},
            {"role": "user", "content": query}
        ],
        stream = True
    )

    response = ''
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

def ask(persona, query):
    
    if persona == 'SNARKY':
        result = ask_snarky(query)
    elif persona == 'HUMBLE':
        result = ask_humble(query)
    else:
        raise ValueError('UNKNOWN')
    
    yield from result

In [ ]:
model_dropdown = gr.Dropdown(choices = ['SNARKY', 'HUMBLE'], value = 'HUMBLE', label = 'Persona')
user_query = gr.Textbox(placeholder = 'Ask your question..', label = 'Chat', autofocus = True, lines = 5)
response_markdown = gr.Markdown(value = "# HELLO", label = 'Response', show_label = True, container = True)

ui = gr.Interface(
    fn = ask,
    inputs = [model_dropdown, user_query],
    outputs = [response_markdown],
    flagging_mode = 'never'
)

ui.launch()